# Setup

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2,3"

In [2]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils import data
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

import re
import time
import pickle

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from collections import Counter

from sklearn import metrics
from sklearn.metrics import classification_report, f1_score

In [3]:
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [40]:
dataset = {'train': [], 'validation': [], 'test': []}
dataset_path = '/data/graphner_embeddings/autoencoder_embeddings_500_2epochs/'

for split in dataset:
    files_list = os.listdir(dataset_path+split)
    for i, filename in tqdm(enumerate(sorted(files_list)), total=len(files_list)):
        dataset[split].append(pickle.load(open(dataset_path+split+'/'+str(i)+'.pickle', 'rb')))

In [7]:
labels = pickle.load(open('labels.pickle', 'rb'))
label2id = {l: i for i, l in enumerate(labels)}
print(label2id)

{'LOC': 0, 'MISC': 1, 'O': 2, 'ORG': 3, 'PER': 4}


# Dataset

In [8]:
x = list(zip(*dataset[split][:10]))
x[0]

(array([-0.01436732,  0.04050591, -0.00089871,  0.00945008,  0.02446665,
        -0.01244789,  0.01709336, -0.02142077,  0.01187329,  0.01046528,
        -0.021883  , -0.00136595,  0.00600611, -0.01329236,  0.00486688,
         0.02181063, -0.00594544, -0.02839023,  0.02226341,  0.0151037 ,
        -0.0096951 , -0.02007345, -0.00057336, -0.01848148,  0.00721815,
         0.00165754,  0.04607863,  0.06198069, -0.02297092,  0.02197368,
         0.01691943,  0.01164624,  0.00712457, -0.00355437,  0.01439966,
        -0.00907129,  0.00530745, -0.00834707, -0.0026083 ,  0.01280503,
         0.00142022,  0.02363534, -0.03480102, -0.01278157,  0.00336421,
         0.00258736,  0.02497259,  0.00027705, -0.05701809, -0.01597444,
         0.00752685,  0.05145542,  0.01890014,  0.03247196,  0.01299692,
        -0.01778115, -0.00883749, -0.01173777, -0.05013476,  0.01755121,
         0.00861797,  0.00312549, -0.02273303,  0.02541528, -0.05910432,
        -0.0291328 ,  0.00334541,  0.04204424, -0.0

In [41]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, dataset, split, label2id=label2id):
        X, Y = zip(*dataset[split])

        self.X = [torch.tensor(x) for x in X]
        self.Y = [torch.tensor(y) for y in Y]
        self.X_len = len(X)
        self.labels = sorted(label2id.keys())
        self.label2id = label2id

    def __len__(self):
        return self.X_len

    def __getitem__(self, index):
        x = self.X[index]
        y = self.Y[index]
        
        return x, y

    def labels(self):
        return self.labels
    
    def Y(self):
        return self.Y

In [12]:
train_set = Dataset(dataset, 'train')
dev_set = Dataset(dataset, 'validation')
test_set = Dataset(dataset, 'test')

In [13]:
batch_size = 256
num_workers = 4

train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, num_workers=num_workers, shuffle=True)
dev_loader = torch.utils.data.DataLoader(dev_set, batch_size=batch_size, num_workers=num_workers, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, num_workers=num_workers, shuffle=False)

In [14]:
input_dim = 0
for local_features, local_labels in train_loader:
    input_dim = local_features.shape[1]
    print(local_features.shape)
    print(local_labels.shape)
    break

torch.Size([256, 100])
torch.Size([256])


In [15]:
input_dim

100

In [16]:
training_counter = Counter([y.item() for y in train_set.Y])
print(training_counter)

Counter({2: 144631, 4: 11124, 3: 9984, 0: 8288, 1: 4583})


In [17]:
labels

['LOC', 'MISC', 'O', 'ORG', 'PER']

# The Model

In [30]:
def backprop(batch_X, batch_Y, model, optimizer, loss_fn):
    Y_hat = model(batch_X)
    loss = loss_fn(Y_hat, batch_Y)
    loss.backward()
    optimizer.step()
    
    return loss.item()

class FeedForwardNetwork(nn.Module):
    def __init__(self, input_dim=input_dim, hidden_dim=1024, output_dim=5, dropout_rate=0.2):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fch = nn.Linear(hidden_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()

        # extra layers layers
        self.dropout = nn.Dropout(p=dropout_rate)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fch(x)
        x = self.relu(x)
        x = self.dropout(x)
        logits = self.fc2(x)

        return logits

In [31]:
ffnet = FeedForwardNetwork(dropout_rate=0.2).to(device)

In [32]:
logs = {'loss/train': {}, 'dev': {}}
writer = SummaryWriter(comment='xp5-autoreg-wei2-lr1e3-mom0.9-wd5e4-hd1024-dr0.2-bs64-dim100', log_dir=None,)

In [33]:
label_counter   = Counter([y.item() for y in train_set.Y])
labels_freqs    = [label_counter[label] / sum(label_counter.values()) for label in range(len(labels))]
labels_weights1 = [min(label_counter.values()) / label_counter[label] for label in range(len(labels))]
labels_weights2 = [np.sqrt(min(label_counter.values())) / np.sqrt(label_counter[label]) for label in range(len(labels))]

weights = torch.Tensor(labels_weights2).to(device)
print(weights)

tensor([0.7436, 1.0000, 0.1780, 0.6775, 0.6419], device='cuda:0')


In [34]:
optimizer_params = {'lr': 1e-3, 
                    'momentum': 0.9, 
                    'weight_decay': 5e-4,
                   }

log_interval = int(len(train_loader) / 2)

loss_fn = nn.CrossEntropyLoss(weight=weights)
optimizer = torch.optim.SGD(ffnet.parameters(), **optimizer_params)

In [37]:
%%time
max_epochs = 20

for epoch in range(len(logs['loss/train']), len(logs['loss/train']) + max_epochs):
    
    # Training
    ffnet.train()
    print('Epoch', epoch)
    logs['loss/train'][epoch] = []
    writer.add_scalar("Learning_rate", optimizer_params['lr'], epoch)

    for batch, (batch_X, batch_Y) in enumerate(tqdm(train_loader)):
        # tranfer to GPU
        batch_X, batch_Y = batch_X.float().to(device), batch_Y.to(device)
        optimizer.zero_grad()
        l = backprop(batch_X, batch_Y, ffnet, optimizer, loss_fn)
        logs['loss/train'][epoch].append(l)
        
        if batch % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch * len(batch_X), len(train_loader.dataset),
                100. * batch / len(train_loader), l))
    
    logs['loss/train'][epoch] = np.mean(logs['loss/train'][epoch])
    writer.add_scalar("Loss/train", logs['loss/train'][epoch], epoch)
    print(f'Average loss on epoch {epoch}: {logs["loss/train"][epoch]}')
    
    # Validation
    ffnet.eval()
    with torch.no_grad():
        preds = []
        gt = []
        for batch, (batch_X, batch_Y) in enumerate(tqdm(dev_loader)):
            # Transfer to GPU
            batch_X = batch_X.float().to(device)
            output = nn.Softmax(dim=1)(ffnet(batch_X))
            preds.append(output.cpu())
            gt.append(batch_Y)

        all_out = [np.argmax(l) for batch in preds for l in batch.numpy()]
        all_gt  = [l for batch in gt for l in batch.numpy()]

        print(classification_report(all_out, all_gt, digits=4))

        micro_F1 = metrics.f1_score(all_gt, all_out, average='micro')
        macro_F1 = metrics.f1_score(all_gt, all_out, average='macro')
        weighted_F1 = metrics.f1_score(all_gt, all_out, average='weighted')
        writer.add_scalar("micro_F1/dev", micro_F1, epoch)
        writer.add_scalar("macro_F1/dev", macro_F1, epoch)
        writer.add_scalar("weighted_F1/dev", weighted_F1, epoch)
        logs['dev'][epoch] = (micro_F1, weighted_F1, macro_F1, (all_gt, all_out))

Epoch 20


Train Epoch: 20 [0/178610 (0%)]	Loss: 0.704399
Train Epoch: 20 [89344/178610 (50%)]	Loss: 0.675723

Average loss on epoch 20: 0.7226199455623299



              precision    recall  f1-score   support

           0     0.5258    0.4653    0.4937      2366
           1     0.3047    0.5026    0.3794       768
           2     0.9491    0.9773    0.9630     35290
           3     0.2674    0.2451    0.2558      2252
           4     0.7922    0.5883    0.6752      4224

    accuracy                         0.8689     44900
   macro avg     0.5678    0.5557    0.5534     44900
weighted avg     0.8669    0.8689    0.8658     44900

Epoch 21


Train Epoch: 21 [0/178610 (0%)]	Loss: 0.603451
Train Epoch: 21 [89344/178610 (50%)]	Loss: 0.722879

Average loss on epoch 21: 0.7166048536840346



              precision    recall  f1-score   support

           0     0.5277    0.4720    0.4983      2341
           1     0.3181    0.5088    0.3915       792
           2     0.9484    0.9783    0.9631     35228
           3     0.2626    0.2501    0.2562      2167
           4     0.8135    0.5837    0.6797      4372

    accuracy                         0.8700     44900
   macro avg     0.5741    0.5586    0.5578     44900
weighted avg     0.8691    0.8700    0.8671     44900

Epoch 22


...


              precision    recall  f1-score   support

           0     0.5377    0.4794    0.5069      2349
           1     0.4467    0.4367    0.4417      1296
           2     0.9438    0.9851    0.9640     34817
           3     0.2573    0.2614    0.2593      2031
           4     0.8352    0.5945    0.6946      4407

    accuracy                         0.8717     44900
   macro avg     0.6041    0.5514    0.5733     44900
weighted avg     0.8665    0.8717    0.8667     44900

Epoch 38


Train Epoch: 38 [0/178610 (0%)]	Loss: 0.735322
Train Epoch: 38 [89344/178610 (50%)]	Loss: 0.610247

Average loss on epoch 38: 0.6757562373548661



              precision    recall  f1-score   support

           0     0.5401    0.4734    0.5046      2389
           1     0.4570    0.4199    0.4376      1379
           2     0.9452    0.9836    0.9640     34918
           3     0.2418    0.2644    0.2526      1887
           4     0.8285    0.6006    0.6964      4327

    accuracy                         0.8720     44900
   macro avg     0.6025    0.5484    0.5711     44900
weighted avg     0.8678    0.8720    0.8677     44900

Epoch 39


Train Epoch: 39 [0/178610 (0%)]	Loss: 0.669933
Train Epoch: 39 [89344/178610 (50%)]	Loss: 0.666475

Average loss on epoch 39: 0.673715257414091



              precision    recall  f1-score   support

           0     0.5392    0.4689    0.5016      2408
           1     0.4562    0.4225    0.4387      1368
           2     0.9451    0.9838    0.9641     34908
           3     0.2200    0.2706    0.2427      1678
           4     0.8467    0.5853    0.6921      4538

    accuracy                         0.8722     44900
   macro avg     0.6014    0.5462    0.5678     44900
weighted avg     0.8714    0.8722    0.8688     44900

CPU times: user 1min 33s, sys: 26.6 s, total: 2min
Wall time: 1min 58s


In [38]:
ffnet.eval()
with torch.no_grad():
    preds = []
    gt = []
    for batch, (batch_X, batch_Y) in enumerate(tqdm(test_loader)):
        # Transfer to GPU
        batch_X = batch_X.float().to(device)
        output = nn.Softmax(dim=1)(ffnet(batch_X))
        preds.append(output.cpu())
        gt.append(batch_Y)

    all_out = [np.argmax(l) for batch in preds for l in batch.numpy()]
    all_gt  = [l for batch in gt for l in batch.numpy()]

    print(classification_report(all_out, all_gt, digits=4))


              precision    recall  f1-score   support

           0     0.5979    0.4368    0.5048      2635
           1     0.4488    0.3627    0.4012      1136
           2     0.9341    0.9843    0.9585     30986
           3     0.2331    0.3384    0.2760      1717
           4     0.8431    0.5455    0.6624      4286

    accuracy                         0.8582     40760
   macro avg     0.6114    0.5335    0.5606     40760
weighted avg     0.8597    0.8582    0.8538     40760

